# Example of SVS processing on the command line

This notebook demos the process of fitting a single voxel scan using the command line scripts included in FSL MRS.
### Contents:  
- [1. File conversion using spec2nii](#1.-File-conversion-using-spec2nii)
- [2. Preprocessing in two ways](#2.-Preprocessing-of-the-data)
    - [2.1. Using individual fsl_mrs_proc commands](#2.1-Pre-processing-using-fsl_mrs_proc)
    - [2.2. In one step using the script fsl_mrs_preproc](#2.2-Pre-processing-using-fsl_mrs_preproc)
- [3. Tissue segmentation](#3.-Run-tissue-segmentation)
- [4. Fitting of the resultant spectrum](#4.-Fit-the-data)
- [5. Display the fitting reports](#5.-Display-the-result)

Will Clarke  
June 2020  
University of Oxford

## 1. File conversion using spec2nii

Run spec2nii twix -v to establish the file contents, then using this information run spec2nii twix -e with the appropriate flags to extract the required scans.

The -j option causes a JSON side-car file to be generated. The -q flag suppresses text output.

This dataset uses a modified versions of the cmrr spectro package sequences on a siemens scanner. It has three sets of water reference scans. The first is taged as a phase correction, iscollected at the start of the main suppressed water scan and will be used for eddy current correction.

The second is collected in a separate scan with only the RF portion of the water suppression disabled. This is used for coil combination (and could be used for eddy current correction). The third is collected with the OVS and all aspects of the water suppression disabled. It therefore experiences eddy currents unlike all the other scans. It will be used for final concentration scaling.

In [ ]:
%%bash

spec2nii twix -v -q example_data/meas_MID310_STEAM_metab_FID115673.dat

From the final lines of the output of the first cell we can see that this "twix" file contains two groups of data.  
The first is tagged as "image" and contains 64 repetitions (sets) of 4096 point FIDs collected on 32 channels.  
The second has a single FID on each channel.

We now call spec2nii again specifying the group we want to extract each time. spec2nii will generate a NIfTI file for each repetition (set) with a size of 1x1x1x4096x32.  
  
We repeat this for the water reference scans, extracting just the image data.

In [ ]:
%%bash
spec2nii twix -e image    -f steam_metab_raw -o data -j -q example_data/meas_MID310_STEAM_metab_FID115673.dat
spec2nii twix -e phasecor -f steam_ecc_raw   -o data -j -q example_data/meas_MID310_STEAM_metab_FID115673.dat

spec2nii twix -e image    -f steam_wref_comb_raw  -o data -j -q example_data/meas_MID311_STEAM_wref1_FID115674.dat
spec2nii twix -e image    -f steam_wref_quant_raw -o data -j -q example_data/meas_MID312_STEAM_wref3_FID115675.dat

In [ ]:
%%bash
# Check the content of the output folder
ls data

After conversion we have 64 averages from the suppressed water scan. The "Set" descriptor at the end of the file name arises from the indexing loop used at acquisition time to store the data. This is likely to vary from sequence to sequence, and is the choice of the pulse sequence programmer. Multiple looping indicies might be used for more complex data acquisition strategies.  
  
We also have one ecc scan, and two "Reps" (another looping index) of each of the water reference scans. In this case both Set and Rep are used to indicate transients that should be averaged toghether.

Notice that each NIfTI file is paired witht a JSON sidecar file which contains useful information for further processing.

## 2. Preprocessing of the data

### 2.1 Pre-processing using fsl_mrs_proc
In this section we will preprocess the data using individual subcommands of the fsl_mrs_proc script to preprocess step-by-step. This example could be used as a template to construct your own preprocessing script in either python or another batch processing script (e.g. bash).  


#### Description of steps
1. Take averages of water references used for combination across files
2. Coil combine the metab data, the ecc data and the quantification data using the "comb" data as the reference.
3. Phase and frequency align the data where there are multiple transients.
4. Combine data across those transients (one in each file) by taking the mean.
5. Run eddy current correction using the appropriate reference.
6. In this data an additional FID point is collected before the echo centre. Remove this.
7. Run HLSVD on the data to remove the residual water in the water suppressed data.
6. Phase the data by a single peak as a crude zero-order phase correction.

Note that the "%sx" command is only required in this example notebook and are unecessary on the command line.

In [ ]:
import os
if not os.path.isdir('fsl_mrs_proc'):
    os.mkdir('fsl_mrs_proc')

#1. Combine comb water data across files
%sx fsl_mrs_proc average --file data/steam_wref_comb_raw*.nii.gz --avgfiles --output fsl_mrs_proc --filename steam_wref_comb

#2. Run coil combination on the three sets of data
%sx fsl_mrs_proc coilcombine --file data/steam_metab_raw*.nii.gz --reference fsl_mrs_proc/steam_wref_comb.nii.gz --output fsl_mrs_proc -r 
%sx fsl_mrs_proc coilcombine --file data/steam_wref_quant_raw*.nii.gz --reference fsl_mrs_proc/steam_wref_comb.nii.gz --output fsl_mrs_proc 
%sx fsl_mrs_proc coilcombine --file data/steam_ecc_raw.nii.gz --reference fsl_mrs_proc/steam_wref_comb.nii.gz --output fsl_mrs_proc 

#3. Align averages of water ref and metab data
%sx fsl_mrs_proc align --file fsl_mrs_proc/steam_metab_raw*.nii.gz --ppm 1.8 3.5 --output fsl_mrs_proc -r --filename steam_metab_align --apod 50 
%sx fsl_mrs_proc align --file fsl_mrs_proc/steam_wref_quant_raw*.nii.gz --ppm 4 6 --output fsl_mrs_proc --filename steam_water_align 

#3. Combine data across averages
%sx fsl_mrs_proc average --file fsl_mrs_proc/steam_metab_align*.nii.gz --avgfiles  --output fsl_mrs_proc -r --filename steam_metab_comb 
%sx fsl_mrs_proc average --file fsl_mrs_proc/steam_water_align*.nii.gz --avgfiles --output fsl_mrs_proc --filename steam_wquant_comb 

#5. Run the eddy current correction on the data
%sx fsl_mrs_proc ecc --file fsl_mrs_proc/steam_metab_comb.nii.gz --reference fsl_mrs_proc/steam_ecc_raw.nii.gz  --output fsl_mrs_proc -r --filename steam_metab_comb_ecc 
%sx fsl_mrs_proc ecc --file fsl_mrs_proc/steam_wquant_comb.nii.gz --reference fsl_mrs_proc/steam_wquant_comb.nii.gz --output fsl_mrs_proc --filename steam_wquant_comb_ecc 

#6. Remove the first FID point
%sx fsl_mrs_proc truncate --file fsl_mrs_proc/steam_metab_comb_ecc.nii.gz --points -1 --pos first --output fsl_mrs_proc -r 
%sx fsl_mrs_proc truncate --file fsl_mrs_proc/steam_wquant_comb_ecc.nii.gz --points -1 --pos first --output fsl_mrs_proc 

#7. Run HLSVD on the data
%sx fsl_mrs_proc remove --file fsl_mrs_proc/steam_metab_comb_ecc.nii.gz --output fsl_mrs_proc -r --filename steam_metab_comb_ecc_hlsvd 

#6. Phase the data
%sx fsl_mrs_proc phase --file fsl_mrs_proc/steam_metab_comb_ecc_hlsvd.nii.gz --output fsl_mrs_proc -r --filename metab 
%sx fsl_mrs_proc phase --ppm 4.6 4.7 --file fsl_mrs_proc/steam_wquant_comb_ecc.nii.gz --output fsl_mrs_proc --filename water 

Combine the reports generated by the individual preprocessing commands

In [ ]:
%sx merge_mrs_reports -d CMD_line_test -o fsl_mrs_proc --delete fsl_mrs_proc/*.html 

### 2.2 Pre-processing using fsl_mrs_preproc
In this section we will preprocess the data using a single preprocessing script which encapsulates all of the commands run above. Some of the commands are optional.

In [ ]:
%sx fsl_mrs_preproc --output fsl_mrs_preproc --data data/steam_metab_raw*.nii.gz --reference data/steam_wref_comb_raw*.nii.gz --quant data/steam_wref_quant_raw*.nii.gz --ecc data/steam_ecc_raw.nii.gz --hlsvd --leftshift 1 --t1 example_data/T1_biascorr.nii.gz --overwrite --report 

# You can also run this script without independent quant and ecc data (just 1 water reference)
%sx fsl_mrs_preproc --output fsl_mrs_preproc_simple --data data/steam_metab_raw*.nii.gz --reference data/steam_wref_comb_raw*.nii.gz --hlsvd --leftshift 1 --t1 example_data/T1_biascorr.nii.gz --overwrite --report 

### 3. Run tissue segmentation
Run tissue segmentation on the packaged T1 data and mask using the SVS voxel. Here we provide a (partial) .anat file produced by [fsl_anat](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/fsl_anat) to speed up execution.

This step requires an installation of FSL compatible with fslpy. If you don't want to run this step the fitting step will run using a pre-packaged tissue segmentation file.

In [ ]:
%sx svs_segment -a example_data/T1.anat fsl_mrs_proc/metab.nii.gz

### 4. Fit the data
Run a quick fit using the Newton method (here using the processed data in `fsl_mrs_preproc`)

In [ ]:
# Quick fit 
%sx fsl_mrs --data fsl_mrs_preproc/metab.nii.gz --basis example_data/steam_11ms --output fit_short --metab_groups Mac --overwrite --report --t1  example_data/T1.anat/T1_biascorr.nii.gz  --h2o fsl_mrs_preproc/wref.nii.gz --tissue_frac segmentation.json



Or run a full fit with the MCMC method

In [ ]:
# Full fit 
%sx fsl_mrs --data fsl_mrs_preproc/metab.nii.gz --basis example_data/steam_11ms --output fit_long --algo MH --metab_groups Mac --overwrite --report --t1  example_data/T1.anat/T1_biascorr.nii.gz --h2o fsl_mrs_preproc/wref.nii.gz --tissue_frac segmentation.json


### 5. Display the result
Display the html reports in new tabs


In [ ]:
%%capture
import webbrowser
import os
current_path = os.getcwd()
# generate a URL
url = os.path.join('file:///'+current_path,'fsl_mrs_preproc/mergedReports.html')
webbrowser.open(url)
url = os.path.join('file:///'+current_path,'fit_short/report.html')
webbrowser.open(url)
url = os.path.join('file:///'+current_path,'fit_long/report.html')
webbrowser.open(url)